<a href="https://colab.research.google.com/github/niwanshu16/FDSP2019/blob/master/Churn_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.getcwd()

In [0]:
os.chdir('/content/drive/My Drive/DeepLearning')


In [0]:
os.getcwd()

In [0]:
import pandas as pd
dataset = pd.read_csv('Churn_Modelling.csv')

In [0]:
import numpy as np

In [0]:
'''Geography: France
        Credit Score: 600
        Gender: Male
        Age: 40 years old
        Tenure: 3 years
        Balance: $60000
        Number of Products: 2
        Does this customer have a credit card? Yes
        Is this customer an Active Member: Yes?
        Estimated Salary: $50000'''
predict = [600,'France' , 0 , 40 , 3, 60000 , 2 , 1 , 1 , 50000]
predict = np.array(predict).reshape(1,-1)
predict

In [0]:
def f(x):
    if x=='Male':
        return 0
    else:
        return 1
dataset['Gender'] = dataset['Gender'].map(f)
features = dataset.iloc[:,3:13].values
labels = dataset.iloc[:,-1].values

from sklearn.preprocessing import LabelEncoder , OneHotEncoder
encoder = LabelEncoder()
features[:,1] = encoder.fit_transform(features[:,1])
predict[:,1] = encoder.transform(predict[:,1])
#features[:,2] = encoder.fit_transform(features[:,2])
ohc = OneHotEncoder(categorical_features = [1])
features = ohc.fit_transform(features).toarray()
predict = ohc.transform(predict).toarray()
features = features[:,1:]
predict = predict[:,1:]

# Applying Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features = sc.fit_transform(features)
predict = sc.transform(predict)

In [0]:
predict[0]

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features , labels , test_size = 0.2 , random_state = 0)


In [0]:
import keras
from keras.layers import Dense
from keras.models import Sequential

In [0]:
classifier = Sequential()

In [0]:
features.shape
predict.shape

In [0]:
# Adding the first adding layer
classifier.add(Dense(units = 6 , kernel_initializer = 'uniform' , activation = 'relu' , input_dim =11))
#Adding the second hidden layer
classifier.add(Dense(units = 6 , kernel_initializer = 'uniform' , activation = 'relu'))
# Adding the output layer 
classifier.add(Dense(units = 1 , kernel_initializer = 'uniform' , activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [0]:
classifier.fit(X_train,y_train,batch_size = 10 , epochs = 100)

In [0]:
# Serialize model to json
classifier_json = classifier.to_json()
with open('classifier.json' , 'w') as json_file:
  json_file.write(classifier_json)
 
# Serialize weights to hdf5
classifier.save_weights('classifier.h5')
print('Saved model to disk')

In [0]:
# Load model from disk
from keras.models import model_from_json
json_file = open('classifier.json' , 'r')
loaded_model_json = json_file.read()
json_file.close()

classifier = model_from_json(loaded_model_json)
classifier.load_weights('classifier.h5')

print('Loaded from disk')

In [0]:
ann_pred = classifier.predict(X_test)
ann_pred = (ann_pred>0.5)

In [0]:
from sklearn.metrics import confusion_matrix , accuracy_score
cm = confusion_matrix(y_test , ann_pred)
score_ann = accuracy_score(y_test , ann_pred)

In [0]:
cm

In [0]:
result = classifier.predict(predict)

In [0]:
result
print(score_ann)

In [0]:
#Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    # Adding 1st layer
    classifier.add(Dense(output_dim = 6 , activation='relu' , init = 'uniform', input_dim = 11 ))
    classifier.add(Dropout(0.2))
    # Adding 2nd layer
    classifier.add(Dense(output_dim = 6 , activation='relu' , init = 'uniform' ))
    classifier.add(Dropout(0.2))
    # Adding final layer 
    classifier.add(Dense(output_dim = 1 , init = 'uniform' , activation = 'sigmoid'))
    # Compiling the classifier
    classifier.compile(optimizer = 'adam' , metrics = ['accuracy'] , loss = 'binary_crossentropy')
    return classifier

classifier = KerasClassifier(build_fn = build_classifier , batch_size = 10 , nb_epoch = 100)
accuracies = cross_val_score(estimator = classifier , X = X_train , y = y_train , cv=10 , n_jobs = -1)



In [0]:
accuracies

In [0]:
mean = accuracies.mean()
variance = accuracies.std()

In [0]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout

def build_classifier(optimizer):
    classifier = Sequential()
    # Adding 1st layer
    classifier.add(Dense(output_dim = 6 , activation='relu' , init = 'uniform', input_dim = 11 ))
    classifier.add(Dropout(0.2))
    # Adding 2nd layer
    classifier.add(Dense(output_dim = 6 , activation='relu' , init = 'uniform' ))
    classifier.add(Dropout(0.2))
    # Adding final layer 
    classifier.add(Dense(output_dim = 1 , init = 'uniform' , activation = 'sigmoid'))
    # Compiling the classifier
    classifier.compile(optimizer = 'adam' , metrics = ['accuracy'] , loss = 'binary_crossentropy')
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'nb_epoch' :[100,500] , 
              'optimizer' :['adam' , 'rmsprop'],
              'batch_size' : [25,32] }
gridsearch = GridSearchCV(estimator = classifier , 
                          param_grid = parameters ,
                          scoring = 'accuracy',
                          cv = 10)
gridsearch = gridsearch.fit(X_train,y_train)
best_parameters = gridsearch.best_params_
best_accuracy = gridsearch.best_score_

In [0]:
print(best_parameters)
best_accuracy